In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [3]:
spark: SparkSession = (
            SparkSession.builder.master("spark://spark-master:7077")
            .appName("tft-analyzer")
            .config("spark.driver.extraJavaOptions", "-Duser.timezone=GMT")
            .config("spark.executor.extraJavaOptions", "-Duser.timezone=GMT")
            .config("spark.sql.session.timeZone", "UTC")
            .config(
                "spark.jars.packages",
                "io.delta:delta-core_2.12:2.2.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0,org.apache.spark:spark-avro_2.12:3.3.0",
            )
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
            .config(
                "spark.delta.logStore.class",
                "org.apache.spark.sql.delta.storage.HDFSLogStore",
            )
            .config(
                "spark.sql.catalog.spark_catalog",
                "org.apache.spark.sql.delta.catalog.DeltaCatalog",
            )
            .config(
                "spark.sql.warehouse.dir", "hdfs://namenode:9000/user/hive/warehouse"
            )
            .config(
                "spark.hadoop.javax.jdo.option.ConnectionURL",
                "jdbc:postgresql://hive-metastore-postgresql/metastore",
            )
            .config(
                "spark.hadoop.javax.jdo.option.ConnectionDriverName",
                "org.postgresql.Driver",
            )
            .config("spark.hadoop.javax.jdo.option.ConnectionUserName", "hive")
            .config("spark.hadoop.javax.jdo.option.ConnectionPassword", "hive")
            .config("spark.hadoop.hive.metastore.uris", "thrift://hive-metastore:9083")
            .config("spark.pyspark.python", "python3")
            .enableHiveSupport()
            .getOrCreate()
        )

In [8]:
spark.read.format("delta").table("silver.matches").where(col("outcome") == "win").show()

+---------------+------------------+---------+----------+-----+---------+------------------+--------------------+------------------+-----------------------+-------+
|       match_id|       game_length|gold_left|last_round|level|placement|players_eliminated|               puuid|   time_eliminated|total_damage_to_players|outcome|
+---------------+------------------+---------+----------+-----+---------+------------------+--------------------+------------------+-----------------------+-------+
|EUW1_6698825371|2012.4964599609375|        1|        31|    9|        3|                 1|Io54W8k3YRT-uSGh0...|1906.4945068359375|                    108|    win|
|EUW1_6698825371|2012.4964599609375|       48|        33|    8|        2|                 2|Vrj3An3x0hKHoIK1d...| 2005.926025390625|                    198|    win|
|EUW1_6698825371|2012.4964599609375|        0|        30|    8|        4|                 0|YdIfLWAZVsc-XO1ZO...|    1841.869140625|                     72|    win|
|EUW1_6698